## Patch effectiveness

In [23]:
from os import path
import logging
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from trojan_defender import (experiment, set_root_folder,
                             datasets, set_db_conf, plot,
                             get_db_conf)
from trojan_defender.detect import saliency_ as saliency
from trojan_defender import datasets
from pymongo import MongoClient
import pandas as pd

In [2]:
# config logging
logging.basicConfig(level=logging.INFO)

# matplotlib size
plt.rcParams['figure.figsize'] = (10, 10)

# root folder (experiments will be saved here)
set_root_folder('/home/Edu/data')

# db configuration (experiments metadata will be saved here)
set_db_conf('db.yaml')

In [50]:
conf = get_db_conf()
client = MongoClient(conf['uri'])
con = client[conf['db']][conf['collection']]

group_name = 'exp-mobile'

In [51]:
# baseline
baseline = con.find({'dataset.name': 'MNIST',
                     'group_name': group_name,
                     'metrics.patch_success_rate': {"$exists": False}})

# attacks
attacked = con.find({'dataset.name': 'MNIST',
                     'group_name': group_name,
                     'metrics.patch_success_rate': {"$exists": True}})

In [52]:
baseline_acc = list(baseline)[0]['metrics']['accuracy_score']

In [53]:
baseline_acc

0.9877

In [54]:
def compute_decay(res):
    d = {}
    d['acc_decay'] = baseline_acc - res['metrics']['accuracy_score']
    d['patch_success_rate'] = res['metrics']['patch_success_rate']
    d['fraction'] = res['dataset']['fraction']
    d['proportion'] = res['dataset']['proportion']
    d['type'] = res['dataset']['type_']
    return d

In [55]:
results = [compute_decay(res) for res in attacked]

In [56]:
df = pd.DataFrame(results)

In [57]:
g = df.groupby(by=['fraction', 'proportion', 'type']).mean()

In [58]:
g

acc_decay  patch_success_rate
fraction proportion type                                
0.01     0.001      block   0.001367            0.003141
         0.005      block   0.001400            0.002735
         0.010      block   0.001100            0.003178
0.05     0.001      block   0.004033            0.007169
         0.005      block   0.004500            0.007391
         0.010      block   0.004033            0.262823
0.10     0.001      block   0.008167            0.017923
         0.005      block   0.008933            0.015854
         0.010      block   0.006767            0.748744
0.15     0.001      block   0.013067            0.186733
         0.005      block   0.014567            0.031559
         0.010      block   0.008633            0.828086
0.20     0.001      block   0.022867            0.162528
         0.005      block   0.028367            0.144309
         0.010      block   0.013700            0.898041

In [59]:
g.to_csv('exp-mobile.csv')